In [392]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [393]:
dfs = []
for fname in os.listdir(input_path):
    plex_data = pd.read_csv('{}/{}'.format(input_path, fname),
                            skiprows=8, names=['patient_id', 'type', 'well', 'error',
                                               'HRP2_pg_ml', 'LDH_Pan_pg_ml',
                                               'LDH_Pv_pg_ml', 'CRP_ng_ml'])
    plex_data = plex_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    plex_data['patient_id'] = plex_data['patient_id'].fillna(method='ffill')
    dfs.append(plex_data)
combined = pd.concat(dfs)
combined = combined.loc[~combined['type'].isnull()]
combined = combined.loc[~combined['type'].str.contains('pixel')]
combined = combined.loc[~combined['patient_id'].isnull()]
combined.head()

patient_id                                 type well error  \
0  calibrator (neat)  reduced concentration (replicate 1)   a1   NaN   
1  calibrator (neat)  reduced concentration (replicate 2)   b1   NaN   
2   calibrator (1:3)  reduced concentration (replicate 1)   a2   NaN   
3   calibrator (1:3)  reduced concentration (replicate 2)   b2   NaN   
4   calibrator (1:9)  reduced concentration (replicate 1)   a3   NaN   

  HRP2_pg_ml LDH_Pan_pg_ml LDH_Pv_pg_ml CRP_ng_ml  
0     600.41      11064.05       571.43   9423.69  
1     483.68       9974.46       424.78  11129.36  
2     219.66       3501.87       181.60   3171.26  
3     196.66       3507.22       149.18   3156.21  
4      65.56       1208.19        57.82    990.08

In [394]:
fail_ids = ['pa-013-28', 'pa-064-63', 'pa-082-7', 'pa-105-28', 'pa-106-56']

In [395]:
combined.loc[combined['patient_id'].str.contains('pa-013-28')]

patient_id                   type well error HRP2_pg_ml  \
26         pa-013-28 (neat)  reduced concentration   c5   NaN     111.45   
60  pa-013-28    50x (1:50)  reduced concentration   d5   NaN    7701.74   
28         pa-013-28 (neat)  reduced concentration   e1   NaN     123.73   
29         pa-013-28 (1:50)  reduced concentration   f1   NaN    9945.48   
30       pa-013-28 (1:2500)  reduced concentration   g1   NaN  298772.67   
31     pa-013-28 (1:125000)  reduced concentration   h1   NaN  401669.68   

   LDH_Pan_pg_ml LDH_Pv_pg_ml     CRP_ng_ml  
26         64.09         9.49       2402.83  
60       1023.80       305.23       2194.03  
28         49.64         3.54       1658.42  
29      < 720.50       134.73     < 1337.00  
30    < 36025.00    < 5825.00    < 66850.00  
31  < 1801250.00    667708.74  < 3342500.00

In [396]:
combined.loc[combined['patient_id'].str.contains('pa-064-63')]

patient_id                   type well error   HRP2_pg_ml  \
56      pa-064-63 (neat)  reduced concentration   a9   NaN     > 330.00   
57      pa-064-63 (1:50)  reduced concentration   b9   NaN      1605.35   
32      pa-064-63 (neat)  reduced concentration   e4   NaN       205.16   
33      pa-064-63 (1:50)  reduced concentration   f4   NaN      1243.41   
34    pa-064-63 (1:2500)  reduced concentration   g4   NaN    < 2675.00   
35  pa-064-63 (1:125000)  reduced concentration   h4   NaN  < 133750.00   

   LDH_Pan_pg_ml LDH_Pv_pg_ml     CRP_ng_ml  
56         22.72        11.85         27.58  
57      < 720.50     < 116.50     < 1337.00  
32       < 14.41       < 2.33         27.70  
33      < 720.50     < 116.50     < 1337.00  
34    < 36025.00    < 5825.00    < 66850.00  
35  < 1801250.00    293919.79  < 3342500.00

In [397]:
sevens = combined.loc[combined['patient_id'].str.contains('pa-082-7')]
sevens = sevens.loc[~sevens['patient_id'].str.contains('77')]
sevens.loc[~sevens['patient_id'].str.contains('70')]

patient_id                   type well error   HRP2_pg_ml  \
39         pa-082-7 (neat)  reduced concentration   c9   NaN       154.55   
73  pa-082-7    50x (1:50)  reduced concentration   d9   NaN     13611.88   
24         pa-082-7 (neat)  reduced concentration  a12   NaN       166.16   
25         pa-082-7 (1:50)  reduced concentration  b12   NaN   > 16500.00   
26       pa-082-7 (1:2500)  reduced concentration  c12   NaN     58199.96   
27     pa-082-7 (1:125000)  reduced concentration  d12   NaN  < 133750.00   

   LDH_Pan_pg_ml LDH_Pv_pg_ml     CRP_ng_ml  
39         58.67        10.10       1351.83  
73        925.23       229.35     < 1337.00  
24        103.70         8.19        511.04  
25       1934.25       185.24     < 1337.00  
26    < 36025.00    < 5825.00    < 66850.00  
27  < 1801250.00  < 291250.00  < 3342500.00

In [398]:
combined.loc[combined['patient_id'].str.contains('pa-105-28')]

patient_id                   type well error   HRP2_pg_ml  \
42       pa-105-28 (neat)  reduced concentration   g2   NaN       138.04   
58  pa-105-28  50x (1:50)  reduced concentration   h2   NaN   > 16500.00   
40       pa-105-28 (neat)  reduced concentration   e2   NaN       135.10   
41       pa-105-28 (1:50)  reduced concentration   f2   NaN     12694.87   
42     pa-105-28 (1:2500)  reduced concentration   g2   NaN     49795.09   
43   pa-105-28 (1:125000)  reduced concentration   h2   NaN  < 133750.00   

   LDH_Pan_pg_ml LDH_Pv_pg_ml     CRP_ng_ml  
42         15.62        14.96       < 26.74  
58        818.08       506.24     < 1337.00  
40         26.92         4.69       < 26.74  
41      < 720.50     < 116.50     < 1337.00  
42    < 36025.00      5878.40    < 66850.00  
43  < 1801250.00    400377.74  < 3342500.00

In [399]:
combined.loc[combined['patient_id'].str.contains('pa-106-56')]

patient_id                                 type well error  \
66      pa-106-56 (neat)                reduced concentration   a8   NaN   
67      pa-106-56 (1:50)                reduced concentration   b8   NaN   
68    pa-106-56 (1:2500)  reduced concentration (replicate 1)   c8   NaN   
69    pa-106-56 (1:2500)  reduced concentration (replicate 2)   c9   NaN   
70  pa-106-56 (1:125000)  reduced concentration (replicate 1)   d8   NaN   
71  pa-106-56 (1:125000)  reduced concentration (replicate 2)   d9   NaN   

     HRP2_pg_ml LDH_Pan_pg_ml LDH_Pv_pg_ml   CRP_ng_ml  
66        24.98       < 14.41         8.08     1479.06  
67      < 53.50      < 720.50     < 116.50   < 1337.00  
68    < 2675.00    < 36025.00    < 5825.00    84772.18  
69    < 2675.00    < 36025.00     29632.73    70748.31  
70  < 133750.00  < 1801250.00  < 291250.00  5143351.02  
71  < 133750.00    2626978.27   1143585.91  4938431.66

In [413]:
combined.loc[combined['patient_id'].str.contains('pa-008-70')].sort_values('patient_id')

patient_id                   type well error   HRP2_pg_ml  \
27  pa-008-70 (1:125000)  reduced concentration  h10   NaN  < 133750.00   
26    pa-008-70 (1:2500)  reduced concentration  g10   NaN    < 2675.00   
25      pa-008-70 (1:50)  reduced concentration  f10   NaN       390.90   
24      pa-008-70 (neat)  reduced concentration  e10   NaN       196.41   

   LDH_Pan_pg_ml LDH_Pv_pg_ml     CRP_ng_ml  
27    9547640.13   2527046.34  < 3342500.00  
26     148732.09     28124.30    < 66850.00  
25       3959.14       665.57     < 1337.00  
24         62.32        28.67       < 26.74

In [363]:
def fix_concentrations(df):
    con = df['concentration'].partition(':')[2]
    con = con.partition(')')[0]
    if len(con) != 0:
        return con
    else:
        return '1'

In [364]:
samples_data = combined.loc[combined['patient_id'].str.contains('pa-')]
samples_data = samples_data.drop('type', axis=1)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[0])
samples_data = samples_data.loc[(samples_data['concentration'].str.contains('neat|50|2500|125000|6250000|312500000'))]
samples_data = samples_data.loc[~samples_data['concentration'].str.contains('low volume')]
samples_data = samples_data.loc[~samples_data['well'].isnull()]
samples_data['concentration'] = samples_data.apply(fix_concentrations, axis=1)
samples_data = samples_data.sort_values(['patient_id', 'well'])
samples_data.head()

patient_id well error HRP2_pg_ml LDH_Pan_pg_ml LDH_Pv_pg_ml  CRP_ng_ml  \
33  pa-001-14   c8   NaN     299.48       < 14.41         2.76    7064.05   
67  pa-001-14   d8   NaN    4138.79      < 720.50     < 116.50    3178.09   
29  pa-001-21  c12   NaN   > 330.00         38.94        12.21    2413.65   
63  pa-001-21  d12   NaN    1377.18       1593.58       227.59  < 1337.00   
27  pa-001-28   g2   NaN   > 330.00         29.86        13.90      92.87   

   concentration  
33             1  
67            50  
29             1  
63            50  
27             1

In [365]:
samples_data.tail()

patient_id well error  HRP2_pg_ml LDH_Pan_pg_ml LDH_Pv_pg_ml   CRP_ng_ml  \
89  pa-183-49   f8   NaN    14637.17      < 720.50       365.66   < 1337.00   
26   pa-183-7   e4   NaN   454365.94     131428.70     15859.37  < 66850.00   
27   pa-183-7   f4   NaN  1151591.32    6571435.11   1244894.95  5977663.09   
24   pa-183-7   g6   NaN      139.85        148.18        15.97     7604.30   
58   pa-183-7   h6   NaN     7770.69       2360.87       646.51     7683.81   

   concentration  
89            50  
26          2500  
27        125000  
24             1  
58            50

In [366]:
samples_data['concentration'].unique()

array(['1', '50', '2500', '125000', '6250000', '312500000'], dtype=object)

In [367]:
# threshhold values for various analytes
threshholds = {'HRP2_pg_ml': 330, 'LDH_Pan_pg_ml': 10514,
               'LDH_Pv_pg_ml': 497, 'CRP_ng_ml': 9574}

In [368]:
# constant to apply to the threshhold for different dilutions
dil_constants = {'50': 1, '2500': 50, '125000': 2500,
                 '6250000': 125000, '312500000': 6250000}

In [369]:
# positivity threshholds for various analytes
pos_threshholds = {'HRP2_pg_ml': 2.3, 'LDH_Pan_pg_ml': 47.8,
                   'LDH_Pv_pg_ml': 75.1, 'CRP_ng_ml': np.nan}

In [370]:
# dilution sets for various dilutions
dilution_sets = {'50': ('1', '50', 'fail'), '2500': ('50', '2500', 'fail'),
                 '125000': ('2500', '125000', 'fail'),
                 '6250000': ('125000', '6250000', 'fail'),
                 '312500000': ('6250000', '312500000', 'fail')} 

In [371]:
def return_decisions(low, high, fail='fail'):
    # Columns = [neat_above, neat_below, neat_LLQ, neat_ULQ, NA]
    # Rows = [dil_above, dil_below, dil_LLQ, dil_ULQ, NA]
    HRP2_matrix = np.array([[high, high, high, high, high],
                            [high, low, low, high, fail],
                            [high, low, low, fail, fail],
                            [high, high, fail, high, high],
                            [fail, high, high, fail, fail]])

    other_matrix = np.array([[high, low, low, high, high],
                               [high, low, low, high, fail],
                               [high, low, low, fail, fail],
                               [high, low, fail, high, high],
                               [fail, low, low, fail, fail]])

    # decisions for various analytes
    decisions = {'HRP2_pg_ml': HRP2_matrix, 'LDH_Pan_pg_ml': other_matrix,
                 'LDH_Pv_pg_ml': other_matrix, 'CRP_ng_ml': other_matrix}
    return(decisions)

In [372]:
def run_compare(df, analyte_val, dil_constant):
    above, below, LLQ, ULQ, NA = False, False, False, False, False
    val = df[analyte_val]
    thresh_val = dil_constant * threshholds[analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
        elif float_val > thresh_val:
            above = True
        elif float_val < thresh_val:
            below = True
    except ValueError:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
    finally:
        return(np.array([above, below, LLQ, ULQ, NA]))

In [457]:
# generate an empty list to fill with small dfs, which will be combined
analyte_dfs = []
# run counts for decision on what to keep
for analyte in threshholds.keys():
# for analyte in ['HRP2_pg_ml']:
    patient_dfs = []
    # iterate over patient_ids
    for i in samples_data['patient_id'].unique():
#     for i in ['pa-004', 'pa-005']:
        patient_data = samples_data.loc[samples_data['patient_id'] == i]
        # generate an empty list to fill with dilution dfs, which will be combined
        dil_dfs = []
        # get number of dilutions
        dilution_number = len(dil_constants.keys())
        # set initial best decision to neat (1)
        best_decision = '1'
        if len(patient_data) == (dilution_number + 1):
            # iterate over dilution values
            for max_dilution in dil_constants.keys():
    #         for max_dilution in ['50']:
                # subset to dilutions
                dil_data = patient_data.loc[patient_data['concentration'].isin([best_decision, max_dilution])]
                # create partial function for generating decision vectors
                dil_constant = (int(max_dilution) / int(best_decision)) / 50
                partial_compare = partial(run_compare, analyte_val=analyte, dil_constant=dil_constant)
                # generate decision vectors
                dil_data['decision_vector'] = dil_data.apply(partial_compare, axis=1)
                # pull decision matrix for given analyte and concentrations
                decisions = return_decisions(best_decision, max_dilution)
                decision_matrix = decisions[analyte]
                # construct empty dataframe to hold best values
                best_df = pd.DataFrame(columns=['patient_id', analyte,
                                                '{}_dilution'.format(analyte),
                                                '{}_well'.format(analyte)])
                try:

                    vector_low = dil_data.loc[dil_data['concentration'] == best_decision,
                                              'decision_vector'].item()
                    vector_high = dil_data.loc[dil_data['concentration'] == max_dilution,
                                               'decision_vector'].item()
                    decision = decision_matrix[vector_low, vector_high].item()
                    if decision in [best_decision, max_dilution]:
                        val = dil_data.loc[dil_data['concentration'] == decision,
                                           analyte].item()
                        well = dil_data.loc[dil_data['concentration'] == decision,
                                            'well'].item()
                        best_decision = decision
                    elif decision == 'fail':
                        val = np.nan
                        well = np.nan
                    else:
                        raise ValueError("Unexpected decision value: {}".format(decision))
                    best_df = best_df.append({'patient_id': i, analyte: val,
                                              '{}_dilution'.format(analyte): decision,
                                              '{}_well'.format(analyte): well}, ignore_index=True)
                    best_decision = decision
                    if decision == 'fail':
                        break
                except ValueError:
                    print("ValueError:")
                    print(analyte, max_dilution, i)
            patient_dfs.append(best_df)
        else:
            print("Incorrect number of dilutions:")
            print(analyte, i)
            continue
    patient_df = pd.concat(patient_dfs)
    analyte_dfs.append(patient_df)
final_df = reduce(lambda left, right: pd.merge(left, right, on='patient_id'), analyte_dfs)

Incorrect number of dilutions:
HRP2_pg_ml pa-001-14
Incorrect number of dilutions:
HRP2_pg_ml pa-001-21
Incorrect number of dilutions:
HRP2_pg_ml pa-001-28
Incorrect number of dilutions:
HRP2_pg_ml pa-001-7


C:\Users\lzoeckler\miniconda3\envs\DIAMETER\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Incorrect number of dilutions:
HRP2_pg_ml pa-002-14
Incorrect number of dilutions:
HRP2_pg_ml pa-002-21
Incorrect number of dilutions:
HRP2_pg_ml pa-002-28
Incorrect number of dilutions:
HRP2_pg_ml pa-002-3
Incorrect number of dilutions:
HRP2_pg_ml pa-002-35
Incorrect number of dilutions:
HRP2_pg_ml pa-002-7
Incorrect number of dilutions:
HRP2_pg_ml pa-003-14
Incorrect number of dilutions:
HRP2_pg_ml pa-003-21
Incorrect number of dilutions:
HRP2_pg_ml pa-003-28
Incorrect number of dilutions:
HRP2_pg_ml pa-003-35
Incorrect number of dilutions:
HRP2_pg_ml pa-003-42
Incorrect number of dilutions:
HRP2_pg_ml pa-003-49
Incorrect number of dilutions:
HRP2_pg_ml pa-003-7
Incorrect number of dilutions:
HRP2_pg_ml pa-004-14
Incorrect number of dilutions:
HRP2_pg_ml pa-004-21
Incorrect number of dilutions:
HRP2_pg_ml pa-004-28
Incorrect number of dilutions:
HRP2_pg_ml pa-004-3
Incorrect number of dilutions:
HRP2_pg_ml pa-004-35
Incorrect number of dilutions:
HRP2_pg_ml pa-004-42
Incorrect number

Incorrect number of dilutions:
HRP2_pg_ml pa-020-49
Incorrect number of dilutions:
HRP2_pg_ml pa-020-56
Incorrect number of dilutions:
HRP2_pg_ml pa-020-63
Incorrect number of dilutions:
HRP2_pg_ml pa-020-7
Incorrect number of dilutions:
HRP2_pg_ml pa-020-70
Incorrect number of dilutions:
HRP2_pg_ml pa-020-77
Incorrect number of dilutions:
HRP2_pg_ml pa-020-84
Incorrect number of dilutions:
HRP2_pg_ml pa-020-91
Incorrect number of dilutions:
HRP2_pg_ml pa-020-98
Incorrect number of dilutions:
HRP2_pg_ml pa-023-14
Incorrect number of dilutions:
HRP2_pg_ml pa-023-21
Incorrect number of dilutions:
HRP2_pg_ml pa-023-28
Incorrect number of dilutions:
HRP2_pg_ml pa-023-35
Incorrect number of dilutions:
HRP2_pg_ml pa-023-42
Incorrect number of dilutions:
HRP2_pg_ml pa-023-49
Incorrect number of dilutions:
HRP2_pg_ml pa-023-56
Incorrect number of dilutions:
HRP2_pg_ml pa-023-63
Incorrect number of dilutions:
HRP2_pg_ml pa-023-7
Incorrect number of dilutions:
HRP2_pg_ml pa-023-70
Incorrect numb

Incorrect number of dilutions:
HRP2_pg_ml pa-041-14
Incorrect number of dilutions:
HRP2_pg_ml pa-041-21
Incorrect number of dilutions:
HRP2_pg_ml pa-041-28
Incorrect number of dilutions:
HRP2_pg_ml pa-041-35
Incorrect number of dilutions:
HRP2_pg_ml pa-041-42
Incorrect number of dilutions:
HRP2_pg_ml pa-041-49
Incorrect number of dilutions:
HRP2_pg_ml pa-041-56
Incorrect number of dilutions:
HRP2_pg_ml pa-041-63
Incorrect number of dilutions:
HRP2_pg_ml pa-041-7
Incorrect number of dilutions:
HRP2_pg_ml pa-041-70
Incorrect number of dilutions:
HRP2_pg_ml pa-041-77
Incorrect number of dilutions:
HRP2_pg_ml pa-041-84
Incorrect number of dilutions:
HRP2_pg_ml pa-041-91
Incorrect number of dilutions:
HRP2_pg_ml pa-042-14
Incorrect number of dilutions:
HRP2_pg_ml pa-042-21
Incorrect number of dilutions:
HRP2_pg_ml pa-042-35
Incorrect number of dilutions:
HRP2_pg_ml pa-042-42
Incorrect number of dilutions:
HRP2_pg_ml pa-042-49
Incorrect number of dilutions:
HRP2_pg_ml pa-042-56
Incorrect num

...

ValueError:
CRP_ng_ml 50 pa-082-7
ValueError:
CRP_ng_ml 2500 pa-082-7
ValueError:
CRP_ng_ml 125000 pa-082-7
ValueError:
CRP_ng_ml 6250000 pa-082-7
ValueError:
CRP_ng_ml 312500000 pa-082-7
Incorrect number of dilutions:
CRP_ng_ml pa-082-70
Incorrect number of dilutions:
CRP_ng_ml pa-082-77
Incorrect number of dilutions:
CRP_ng_ml pa-083
Incorrect number of dilutions:
CRP_ng_ml pa-083-14
Incorrect number of dilutions:
CRP_ng_ml pa-083-7
Incorrect number of dilutions:
CRP_ng_ml pa-084-14
Incorrect number of dilutions:
CRP_ng_ml pa-084-21
Incorrect number of dilutions:
CRP_ng_ml pa-084-28
Incorrect number of dilutions:
CRP_ng_ml pa-084-35
Incorrect number of dilutions:
CRP_ng_ml pa-084-42
Incorrect number of dilutions:
CRP_ng_ml pa-084-49
Incorrect number of dilutions:
CRP_ng_ml pa-084-56
Incorrect number of dilutions:
CRP_ng_ml pa-084-63
Incorrect number of dilutions:
CRP_ng_ml pa-084-7
Incorrect number of dilutions:
CRP_ng_ml pa-084-70
Incorrect number of dilutions:
CRP_ng_ml pa-084-77
I

Incorrect number of dilutions:
CRP_ng_ml pa-105-14
Incorrect number of dilutions:
CRP_ng_ml pa-105-21
ValueError:
CRP_ng_ml 50 pa-105-28
ValueError:
CRP_ng_ml 2500 pa-105-28
ValueError:
CRP_ng_ml 125000 pa-105-28
ValueError:
CRP_ng_ml 6250000 pa-105-28
ValueError:
CRP_ng_ml 312500000 pa-105-28
Incorrect number of dilutions:
CRP_ng_ml pa-105-35
Incorrect number of dilutions:
CRP_ng_ml pa-105-42
Incorrect number of dilutions:
CRP_ng_ml pa-105-49
Incorrect number of dilutions:
CRP_ng_ml pa-105-56
Incorrect number of dilutions:
CRP_ng_ml pa-105-63
Incorrect number of dilutions:
CRP_ng_ml pa-105-7
Incorrect number of dilutions:
CRP_ng_ml pa-105-70
Incorrect number of dilutions:
CRP_ng_ml pa-106-14
Incorrect number of dilutions:
CRP_ng_ml pa-106-21
Incorrect number of dilutions:
CRP_ng_ml pa-106-28
Incorrect number of dilutions:
CRP_ng_ml pa-106-3
Incorrect number of dilutions:
CRP_ng_ml pa-106-35
Incorrect number of dilutions:
CRP_ng_ml pa-106-42
Incorrect number of dilutions:
CRP_ng_ml pa-

Incorrect number of dilutions:
CRP_ng_ml pa-122-14
Incorrect number of dilutions:
CRP_ng_ml pa-122-21
Incorrect number of dilutions:
CRP_ng_ml pa-122-28
Incorrect number of dilutions:
CRP_ng_ml pa-122-35
Incorrect number of dilutions:
CRP_ng_ml pa-122-7
Incorrect number of dilutions:
CRP_ng_ml pa-123-14
Incorrect number of dilutions:
CRP_ng_ml pa-123-21
Incorrect number of dilutions:
CRP_ng_ml pa-123-28
Incorrect number of dilutions:
CRP_ng_ml pa-123-35
Incorrect number of dilutions:
CRP_ng_ml pa-123-7
Incorrect number of dilutions:
CRP_ng_ml pa-124-14
Incorrect number of dilutions:
CRP_ng_ml pa-124-21
Incorrect number of dilutions:
CRP_ng_ml pa-124-28
Incorrect number of dilutions:
CRP_ng_ml pa-124-35
Incorrect number of dilutions:
CRP_ng_ml pa-124-42
Incorrect number of dilutions:
CRP_ng_ml pa-124-49
Incorrect number of dilutions:
CRP_ng_ml pa-124-56
Incorrect number of dilutions:
CRP_ng_ml pa-124-63
Incorrect number of dilutions:
CRP_ng_ml pa-124-7
Incorrect number of dilutions:
CRP

Incorrect number of dilutions:
CRP_ng_ml pa-140-14
Incorrect number of dilutions:
CRP_ng_ml pa-140-21
Incorrect number of dilutions:
CRP_ng_ml pa-140-28
Incorrect number of dilutions:
CRP_ng_ml pa-140-35
Incorrect number of dilutions:
CRP_ng_ml pa-140-42
Incorrect number of dilutions:
CRP_ng_ml pa-140-49
Incorrect number of dilutions:
CRP_ng_ml pa-140-56
Incorrect number of dilutions:
CRP_ng_ml pa-140-63
Incorrect number of dilutions:
CRP_ng_ml pa-140-7
Incorrect number of dilutions:
CRP_ng_ml pa-140-70
Incorrect number of dilutions:
CRP_ng_ml pa-140-77
Incorrect number of dilutions:
CRP_ng_ml pa-140-84
Incorrect number of dilutions:
CRP_ng_ml pa-140-91
Incorrect number of dilutions:
CRP_ng_ml pa-140-98
Incorrect number of dilutions:
CRP_ng_ml pa-144-14
Incorrect number of dilutions:
CRP_ng_ml pa-144-21
Incorrect number of dilutions:
CRP_ng_ml pa-144-28
Incorrect number of dilutions:
CRP_ng_ml pa-144-35
Incorrect number of dilutions:
CRP_ng_ml pa-144-42
Incorrect number of dilutions:
C

Incorrect number of dilutions:
CRP_ng_ml pa-164-14
Incorrect number of dilutions:
CRP_ng_ml pa-164-21
Incorrect number of dilutions:
CRP_ng_ml pa-164-28
Incorrect number of dilutions:
CRP_ng_ml pa-164-35
Incorrect number of dilutions:
CRP_ng_ml pa-164-42
Incorrect number of dilutions:
CRP_ng_ml pa-164-49
Incorrect number of dilutions:
CRP_ng_ml pa-164-56
Incorrect number of dilutions:
CRP_ng_ml pa-164-63
Incorrect number of dilutions:
CRP_ng_ml pa-164-7
Incorrect number of dilutions:
CRP_ng_ml pa-164-70
Incorrect number of dilutions:
CRP_ng_ml pa-164-77
Incorrect number of dilutions:
CRP_ng_ml pa-164-84
Incorrect number of dilutions:
CRP_ng_ml pa-164-91
Incorrect number of dilutions:
CRP_ng_ml pa-164-98
Incorrect number of dilutions:
CRP_ng_ml pa-165-14
Incorrect number of dilutions:
CRP_ng_ml pa-165-21
Incorrect number of dilutions:
CRP_ng_ml pa-165-28
Incorrect number of dilutions:
CRP_ng_ml pa-165-35
Incorrect number of dilutions:
CRP_ng_ml pa-165-42
Incorrect number of dilutions:
C

In [458]:
final_df

patient_id      HRP2_pg_ml HRP2_pg_ml_dilution HRP2_pg_ml_well  \
0     pa-001-3   1537195777.01           312500000              h3   
1       pa-002  < 334375000.00           312500000             d10   
2       pa-003    < 6687500.00             6250000              g2   
3       pa-004     < 133750.00              125000              h3   
4       pa-005  < 334375000.00           312500000              c1   
5       pa-007  < 334375000.00           312500000              h2   
6       pa-008  < 334375000.00           312500000              h4   
7       pa-009    < 6687500.00             6250000              g4   
8       pa-010    < 6687500.00             6250000              g8   
9       pa-011  < 334375000.00           312500000              h6   
10      pa-012  < 334375000.00           312500000              h7   
11      pa-013  < 334375000.00           312500000              h8   
12      pa-014    < 6687500.00             6250000             c12   
13      pa-015    < 6687500.00             6250000              e2   
14      pa-016    < 6687500.00             6250000              e6   
15      pa-017    < 6687500.00             6250000              b1   
16      pa-020     < 133750.00              125000              d3   
17      pa-023    < 6687500.00             6250000              e4   
18      pa-024    < 6687500.00             6250000              e6   
19      pa-026     < 133750.00              125000              f7   
20      pa-028    < 6687500.00             6250000              e4   
21      pa-030  < 334375000.00           312500000              f8   
22      pa-032     < 133750.00              125000              f9   
23      pa-034    < 6687500.00             6250000              g6   
24      pa-035    < 6687500.00             6250000              g8   
25      pa-036    < 6687500.00             6250000              d3   
26      pa-037    < 6687500.00             6250000             g10   
27      pa-038  < 334375000.00           312500000              a9   
28      pa-039  < 334375000.00           312500000              a7   
29      pa-040    < 6687500.00             6250000              c4   
..         ...             ...                 ...             ...   
97      pa-133    < 6687500.00             6250000              g3   
98      pa-134    < 6687500.00             6250000              g4   
99      pa-135    < 6687500.00             6250000              g9   
100     pa-136    < 6687500.00             6250000             g10   
101     pa-137    < 6687500.00             6250000              c8   
102     pa-138    < 6687500.00             6250000              c9   
103     pa-139    < 6687500.00             6250000             c12   
104     pa-140  < 334375000.00           312500000             d12   
105     pa-144  < 334375000.00           312500000             d10   
106     pa-145     < 133750.00              125000             b11   
107     pa-148  < 334375000.00           312500000              h6   
108     pa-149  < 334375000.00           312500000              h7   
109     pa-150    < 6687500.00             6250000             g11   
110     pa-151    < 6687500.00             6250000             g12   
111     pa-152    < 6687500.00             6250000             c10   
112     pa-153    < 6687500.00             6250000             c11   
113     pa-154    < 6687500.00             6250000              g2   
114     pa-157    < 6687500.00             6250000              g5   
115     pa-158    < 6687500.00             6250000              g3   
116     pa-159    < 6687500.00             6250000              g4   
117     pa-160     < 133750.00              125000              f5   
118     pa-162  < 334375000.00           312500000              h9   
119     pa-163  < 334375000.00           312500000             h10   
120     pa-164  < 334375000.00           312500000             h11   
121     pa-165  < 334375000.00           312500000             h12   
122     pa-166